In [2]:
import warnings
from collections import namedtuple
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils import load_state_dict_from_url

In [3]:
__all__ = ['GoogLeNet', 'googlenet']

model_urls = {
    # GoogLeNet ported from TensorFlow
    'googlenet': 'https://download.pytorch.org/models/googlenet-1378be20.pth',
}

# 定义一个namedtuple类型 GoogLeNetOutputs ，并包含logits，aux_logits2和aux_logits1属性。
_GoogLeNetOutputs = namedtuple('GoogLeNetOutputs', ['logits', 'aux_logits2', 'aux_logits1'])

In [16]:
ex_namedtuple = _GoogLeNetOutputs('a','b','c')
print(ex_namedtuple)

GoogLeNetOutputs(logits='a', aux_logits2='b', aux_logits1='c')


In [6]:
# 提前定义一些需要的module
class Inception(nn.Module):         #这就是Inception V1中精心设计的Inception Module,输入(b,in_channels,l,w)

    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):      
        super(Inception, self).__init__()

        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)       # 卷积是1*1大小的，纯减小channel数,最后图像大小不变

        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1),               # 纯减小channel数
            BasicConv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)           # channel数目稍微增大，并且卷积是3*3的，想要提取更多信息
        )                                                                    # 最后图像大小不变

        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1),                # 纯减小channel数
            BasicConv2d(ch5x5red, ch5x5, kernel_size=3, padding=1)            # channel数目稍微增大，并且卷积是3*3的，想要提取更多信息
        )                                                                     # 最后图像大小不变

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),  #图像大小不变
            BasicConv2d(in_channels, pool_proj, kernel_size=1)                # 纯减小channel数
        )                                                                     # 最后图像大小不变

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)     #channel通道数叠加 ，最后输出尺寸为（b,ch1x1+ch3x3+ch5x5+pool_proj）


class InceptionAux(nn.Module):

    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.conv = BasicConv2d(in_channels, 128, kernel_size=1)

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = F.adaptive_avg_pool2d(x, (4, 4))
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = x.view(x.size(0), -1)
        # N x 2048
        x = F.relu(self.fc1(x), inplace=True)
        # N x 2048
        x = F.dropout(x, 0.7, training=self.training)
        # N x 2048
        x = self.fc2(x)
        # N x 1024

        return x


class BasicConv2d(nn.Module):       #这里一个基本的conv2d包含了conv2d，bn，relu

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [7]:
def googlenet(pretrained=False, progress=True, **kwargs):
    r"""GoogLeNet (Inception v1) model architecture from
    `"Going Deeper with Convolutions" <http://arxiv.org/abs/1409.4842>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        aux_logits (bool): If True, adds two auxiliary branches that can improve training.
            Default: *False* when pretrained is True otherwise *True*
        transform_input (bool): If True, preprocesses the input according to the method with which it
            was trained on ImageNet. Default: *False*
    """
    if pretrained:       
        if 'transform_input' not in kwargs:
            kwargs['transform_input'] = True
        if 'aux_logits' not in kwargs:
            kwargs['aux_logits'] = False
        if kwargs['aux_logits']:
            warnings.warn('auxiliary heads in the pretrained googlenet model are NOT pretrained, '
                          'so make sure to train them')
        original_aux_logits = kwargs['aux_logits']
        kwargs['aux_logits'] = True
        kwargs['init_weights'] = False
        model = GoogLeNet(**kwargs)
        state_dict = load_state_dict_from_url(model_urls['googlenet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        if not original_aux_logits:
            model.aux_logits = False
            del model.aux1, model.aux2
        return model

    return GoogLeNet(**kwargs)



class GoogLeNet(nn.Module):
    def __init__(self, num_classes=1000, aux_logits=True, transform_input=False, init_weights=True):
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits
        self.transform_input = transform_input

        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)  #(b,3,224,224) -> (b,64,112,112)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, ceil_mode=True)            #(b,64,112,112) -> (b,64,56,56)
        self.conv2 = BasicConv2d(64, 64, kernel_size=1)                      #(b,64,56,56) -> (b,64,56,56)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)          #(b,64,56,56) -> (b,192,56,56)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)            #(b,192,56,56) -> (b,192,28,28)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)           #(b,192,28,28) -> (b,256,28,28)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)         #(b,256,28,28) -> (b,480,28,28)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)            #(b,480,28,28) -> (b,480,14,14)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)          #(b,480,28,28) -> (b,512,14,14)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)         #(b,512,14,14) -> (b,512,14,14)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)         #(b,512,14,14) -> (b,512,14,14)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)         #(b,512,14,14) -> (b,528,14,14)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)       #(b,528,14,14) -> (b,832,14,14)
        self.maxpool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)            #(b,832,14,14) -> (b,832,7,7)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)       #(b,832,7,7) -> (b,832,7,7)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)       #(b,832,14,14) -> (b,1024,7,7)

        if aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(1024, num_classes)

        if init_weights:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                import scipy.stats as stats
                X = stats.truncnorm(-2, 2, scale=0.01)
                values = torch.as_tensor(X.rvs(m.weight.numel()), dtype=m.weight.dtype)
                values = values.view(m.weight.size())
                with torch.no_grad():
                    m.weight.copy_(values)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        if self.transform_input:
            x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5           #给指定位置1加上维数为1的维度
            x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((x_ch0, x_ch1, x_ch2), 1)

        # N x 3 x 224 x 224
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.conv3(x)
        # N x 192 x 56 x 56
        x = self.maxpool2(x)

        # N x 192 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(x)
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        if self.training and self.aux_logits:
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14
        if self.training and self.aux_logits:
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        # N x 832 x 14 x 14
        x = self.maxpool4(x)
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7

        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = x.view(x.size(0), -1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        # N x 1000 (num_classes)
        if self.training and self.aux_logits:
            return _GoogLeNetOutputs(x, aux2, aux1)
        return x
    


In [21]:
my_google_net = googlenet(pretrained=True,progress=True,num_classes=1000,aux_logits=True,transform_input=True,init_weights=True)
print(my_google_net.training)
print(my_google_net.aux_logits)
print(my_google_net.aux1)   #首先，如果aux_logits为true，则这个net肯定包含了aux1和aux2层

D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:18: UserWarning: auxiliary heads in the pretrained googlenet model are NOT pretrained, so make sure to train them


True
True
InceptionAux(
  (conv): BasicConv2d(
    (conv): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1000, bias=True)
)
